# Part 2.1

Reading training and parsing file

In [449]:
def read_file(file_path):
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''): 
            break
        elif(line == '\n'):
            parsed_file.append((None, None))
            continue
        else: 
            temp = line.replace('\n','').split(' ')
            parsed_file.append(tuple(temp)) #return a tuple of word and its tag
      
    return parsed_file

# print(read_file("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"))
'''
returns something like this:
[('We', 'O'), ('were', 'O'), ('then', 'O'), ('charged', 'O'), ('for', 'O')......]
'''
        
        

"\nreturns something like this:\n[('We', 'O'), ('were', 'O'), ('then', 'O'), ('charged', 'O'), ('for', 'O')......]\n"

A helper function, smartify() is a function that takes file_path as input and returns training data in a 'smart' format.
It returns a dictionary smart_data which is basically training data but in a smart format.

Smart format is defined as: {tag:{word:frequency}}
example: {'O': {'We': 83, 'were': 125, 'then': 15, 'charged': 2, 'for': 279}

Also, smart_data.keys() (and also smartify().keys()) is list of all the tags 

In [450]:
def smartify(file_path):
    training_data = read_file(file_path)
    smart_data = {} #smart_data is basically training data but in a smart format 
                    #smart format is {tag:{word:frequency}}
                    #example: {'O': {'We': 83, 'were': 125, 'then': 15, 'charged': 2, 'for': 279}
    for word_tag_pair in training_data:
        
        if word_tag_pair==(None,None): #annoying case -.-
            continue 
        
        #if tag not in smart_data yet, add it 
        if not word_tag_pair[1] in smart_data.keys():
            smart_data[word_tag_pair[1]]={}
        
        # if word not in smart_data[tag] yet, add it
        if not word_tag_pair[0] in smart_data[word_tag_pair[1]].keys():
            smart_data[word_tag_pair[1]][word_tag_pair[0]] = 1
        
        #increase frequency for each word-tag pair if seen
        smart_data[word_tag_pair[1]][word_tag_pair[0]] = smart_data[word_tag_pair[1]][word_tag_pair[0]] +1
    return smart_data
    
# print (smartify("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"))
    
    

A function to modify training set

In [451]:
def modify_training_set(training_data, k):
    #input: smart_data
    count = 0 #not sure if we will need number of words replaced by unk, but lets keep count anyway
    
    remove_words = []
    training_data = training_data
    for tag, word in training_data.items():
        for specific_word, f in list(word.items()):
            if (f<k):
                count +=f
#                 print(specific_word,f)
                #replace with #unk#
                word.pop(specific_word)
                remove_words.append((tag,specific_word))
        word["#UNK#"] = count

    return training_data
# modify_training_set(smartify("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"),3) 

A function to estimate emission parameters



In [452]:
import math
def emission(file_path):
    emissions = {}
    training_data = smartify(file_path)
    training_data = modify_training_set(training_data, k=9) #modify training data 
    
    for tag, word in training_data.items():
#         print(word.values(),"\n")
        count_y = sum(word.values())
#         print (count_y)
        for specific_word, f in word.items():
#             print(specific_word,f)
            temp_tup = (specific_word,tag)
#             print(specific_word, f)
            emissions[temp_tup] = float(f/(count_y))


    return emissions
        

print(emission("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"))

{('We', 'O'): 0.003035559410234172, ('were', 'O'): 0.004553339115351257, ('then', 'O'): 0.0005782017924255566, ('for', 'O'): 0.01011853136744724, ('their', 'O'): 0.001915293437409656, ('most', 'O'): 0.0009757155247181266, ('expensive', 'O'): 0.000505926568372362, ('(', 'O'): 0.0029271465741543797, ('$', 'O'): 0.0016984677652500722, (')', 'O'): 0.003035559410234172, ('when', 'O'): 0.001517779705117086, ('we', 'O'): 0.004842440011564036, ('in', 'O'): 0.009287366290835502, ('fact', 'O'): 0.0003613761202659728, ('a', 'O'): 0.019442035270309337, ('of', 'O'): 0.010732870771899394, ('than', 'O'): 0.001409366869037294, ('half', 'O'): 0.0003613761202659728, ('that', 'O'): 0.006504770164787511, ('price', 'O'): 0.0010479907487713212, ('.', 'O'): 0.055109858340560855, ('is', 'O'): 0.01673171436831454, ('nothing', 'O'): 0.0006504770164787511, ('special', 'O'): 0.0005420641803989592, ('Great', 'O'): 0.0011202659728245157, ('and', 'O'): 0.02768141081237352, ('worth', 'O'): 0.0011564035848511131, ('ev

A function for reading test file

In [453]:
def read_test_file(file_path):
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''):
            break
        if(line == '\n'):
            parsed_file.append(None)
            continue
        parsed_file.append(line.replace('\n',''))
    return parsed_file

# print(read_test_file("C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"))
    

A function to clean up the train file (replace words not in training set with '#UNK#')

In [454]:
def clean_up_test_data(test_data, emissions_dict):
    
    test_data = test_data
#     print("Original test data:",len(test_data))
    
    training_data = []
    list_of_replaced_words = []
    for tup in emissions_dict.keys():
        training_data.append(tup[0])

    #compare training and test data and replace non-common words with #UNK#
    for word in test_data:
        if word =='START' or word=='STOP':
            continue
        elif word not in training_data:
            list_of_replaced_words.append(word) #we might need this for testing if the function is working
            index = test_data.index(word)
            test_data[index] = "#UNK#"


    return test_data
# print (clean_up_test_data( read_test_file("C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"),emission("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train")))

In [455]:
def construct_sentences(file_path):
    file = open(file_path,'r',encoding = "utf8")
    sentences = []
    sentence = ['START']
    for line in file:
        if(line == ''):
            break
        if(line == '\n'):
            sentence.append('STOP')
            sentences.append(sentence)
            sentence = ['START']
            continue
        else:
            sentence.append(line.replace('\n',''))
            
    return sentences 

# print(construct_sentences("C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"))
    

Simple sentiment analysis function that computes argmax

In [456]:
def sentiment_anal(train_file_path, test_file_path):
#     test_data = read_test_file(test_file_path)
    emissions = emission(train_file_path)

    sentences = construct_sentences(test_file_path)
    for i in range(len(sentences)):
        sentences[i] = clean_up_test_data(sentences[i], emissions)

    sentences_zipped = []
    for sentence in sentences:
        argmax = []
        for word in sentence:
            #         print(word)
            if word == None:
                argmax.append((None, None))
                continue
            temp = {}

            # filter out common words into temp
            for key in emissions.keys():

                if (key[0] == word):
                    #                 print(emissions[key])
                    temp[key] = emissions[key]

            if temp == {}:
                continue
            else:
                result = max(temp, key=temp.get)
            argmax.append(result)
        sentences_zipped.append(argmax)
    return sentences_zipped


# sentiment_anal("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train",
#                "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in")

Write into dev.pn.out

In [457]:
def write_dev_out(input_data, output_path):
    path = output_path
    filename = "dev.p22.aish.out"
    with open(path + filename, 'w', encoding="utf8") as f:
        for i in range(len(input_data)):
            if i != 0:
                f.write("\n")
            for tup in input_data[i]:
                if (tup[0] == 'START' or tup[0] == 'STOP'):
                    pass
                else:
                    f.write(tup[0] + ' ' + tup[1] + '\n')

    f.close()


output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"
write_dev_out(sentiment_anal(training_file,testing_file),output_path)

output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\FR\\"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\FR\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\FR\\dev.in"
write_dev_out(sentiment_anal(training_file,testing_file),output_path)

output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\CN\\"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\CN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\CN\\dev.in"
write_dev_out(sentiment_anal(training_file,testing_file),output_path)

output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\SG\\"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\SG\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\SG\\dev.in"
write_dev_out(sentiment_anal(training_file,testing_file),output_path)

# Part 3

Function to take training data as input

In [458]:
def prepare_input_data_for_transition_paramaters(file_path):
    input_data = (read_file(file_path))
    input_data = [("START",None)]+input_data
    
    #wherever (None, None) is seen, instert a STOP and START
    for tup in input_data:
        if (tup == (None,None)):
            index = input_data.index(tup)
            #add a stop
            input_data[index] = ("STOP",None)
            input_data = input_data[:index+1] + [("START", None)]+ input_data[index+1:]
            
    #take care of the ending of the file
    input_data.pop(-2)
    return input_data 
    
    
# prepare_input_data_for_transition_paramaters("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train")

In [459]:
def estimate_transition_parameters(file_path):
    input_data = prepare_input_data_for_transition_paramaters(file_path)
    
    emission_pairs = {}
    yi_1_count = {}
    
    
    for i in range (len(input_data)-1):
        yi_1 = input_data[i][1]
        yi = input_data[i+1][1]
        
        temp_tup = (yi_1,yi)
        if not temp_tup in emission_pairs.keys():
            emission_pairs[temp_tup] = 1
        else:
            emission_pairs[temp_tup]+=1
        
        if not yi_1 in yi_1_count.keys():
            yi_1_count[yi_1] = 1
        else: 
            yi_1_count[yi_1] += 1
            
            
#     print (emission_pairs)
#     print (yi_1_count)

    transition_params = emission_pairs #this is just for convenience
    
    for yi_1, count in yi_1_count.items():
        
        for tup in emission_pairs.keys():
            if (tup[0]==yi_1):
                transition_params[tup] = float(emission_pairs[tup]/count)
    
#     print(transition_params)
    return(transition_params)
           
    
# estimate_transition_parameters("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train")

VITERBBBEEEEEHHHHHHHHHHHHHHHHHHHHH

In [460]:
def construct_sentences(file_path):
    file = open(file_path)
    sentences = []
    sentence = ['START']
    for line in file:
        if(line == ''):
            break
        if(line == '\n'):
            sentence.append('STOP')
            sentences.append(sentence)
            sentence = ['START']
            continue
        else:
            sentence.append(line.replace('\n',''))
            
    return sentences 

# print(construct_sentences("C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"))
    

In [461]:
import math
import random
def viterbi(train_file_path, test_file_path):
    
    tags = ['O','B-positive','B-neutral','B-negative','I-positive','I-negative','I-neutral']
    emission_p = emission(train_file_path) #{('We', 'O'): 0.003423668687868663, ('were', 'O'): 0.005156127541970878, ('then', 'O'): 0.0006187353050365054,}
    transition_p = estimate_transition_parameters(train_file_path) #{(None, 'O'): 0.47022696929238983, ('O', 'O'): 0.8601188020790363, ('O', None): 0.07623133404834585,}
    sentences = construct_sentences(test_file_path)
    predictions = []
    for i in range(len(sentences)):
        sentences[i]=clean_up_test_data(sentences[i],emission_p)
    
    for sentence in sentences:
        dp_table = [{},{}]
        for tag in tags:
            if((None,tag) in transition_p.keys()):                
                a_i_j = transition_p[(None,tag)] #transition probability from 'START'to tag
            else:
                a_i_j = 0
                            
            if( (sentence[1],tag) in emission_p.keys()):                
                b_j = emission_p[(sentence[1],tag)] #probability that tag emits word
            else:
                b_j = 0
            dp_table[1][tag] = {'p': a_i_j*b_j, 'previous':None}

        for t in range(2,len(sentence)-1):
            dp_table.append({})
            transition_probabilities = {}

            for tag in tags:
                for prev_tag in tags:

                    if((prev_tag,tag) in transition_p.keys()):                           
                        a_i_j = transition_p[(prev_tag,tag)] #transition probability from 'START'to tag
                    else:
                        a_i_j = 0

                    if(prev_tag in dp_table[t-1].keys()):
                        t_p = dp_table[t-1][prev_tag]['p']*a_i_j
                        transition_probabilities[(prev_tag,tag)]=t_p

                max_t_p = max(transition_probabilities.values())
                
                for key in transition_probabilities.keys():
                    if(transition_probabilities[key]==max_t_p):
                        argmax_t_p = key
                
                if((sentence[t],tag) in emission_p.keys()):
                    if(sentence[t]=="#UNK#"):
                        tag = 'O'
                        
                    b_j = emission_p[(sentence[t],tag)] #probability that tag emits word
                else:
                    b_j = 0
                dp_table[t][tag] = {'p':max_t_p*b_j, 'previous':argmax_t_p[0]}

        opt = []
        pmax = max(probabilities['p'] for probabilities in dp_table[-1].values())
        backward_previous = None
        for tag, data in dp_table[-1].items():
            if data["p"] == pmax:
                opt.append(tag)
                prev_tag = tag
                break
        
        for t in range(len(dp_table) - 2, -1, -1):
            opt.insert(0, dp_table[t + 1][prev_tag]["previous"])
            prev_tag = dp_table[t + 1][prev_tag]["previous"]
        
        opt.append(None) #to represent STOP (adding this for convenience and clarity)
        predictions.append(list(zip(sentence,opt)))
        
    return(predictions)
        
        
        
viterbi("C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train" ,"C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in")
    
    

[[('START', None),
  ('When', 'O'),
  ('I', 'O'),
  ('#UNK#', 'O'),
  ('this', 'O'),
  ('#UNK#', 'O'),
  (',', 'O'),
  ('I', 'O'),
  ("didn't", 'O'),
  ('think', 'O'),
  ('I', 'O'),
  ('would', 'O'),
  ('be', 'O'),
  ('#UNK#', 'O'),
  ('to', 'O'),
  ('get', 'O'),
  ('in', 'O'),
  ('at', 'O'),
  ('#UNK#', 'O'),
  (',', 'O'),
  ('but', 'O'),
  ('I', 'O'),
  ('was', 'O'),
  ('#UNK#', 'O'),
  ('to', 'O'),
  ('get', 'O'),
  ('in', 'O'),
  (',', 'O'),
  ('#UNK#', 'O'),
  ('with', 'O'),
  ('#UNK#', 'O'),
  ('other', 'O'),
  ('#UNK#', 'O'),
  ('.', 'O'),
  ('STOP', None)],
 [('START', None),
  ('The', 'O'),
  ('#UNK#', 'O'),
  ('and', 'O'),
  ('#UNK#', 'O'),
  ('#UNK#', 'O'),
  ('are', 'O'),
  ('excellent', 'O'),
  ('.', 'O'),
  ('STOP', None)],
 [('START', None),
  ('My', 'O'),
  ('#UNK#', 'O'),
  ('pizza', 'B-positive'),
  ('#UNK#', 'O'),
  ('in', 'O'),
  ('#UNK#', 'O'),
  ('STOP', None)],
 [('START', None),
  ('Food', 'B-positive'),
  ('#UNK#', 'O'),
  (',', 'O'),
  ('its', 'O'),
  ('ok', '

In [462]:
def write_dev_out(input_data, output_path):
    path = output_path
    filename = "dev.p33.aish.out"
    with open(path + filename, 'w',encoding = "utf8") as f:
        for i in range(len(input_data)):
            if i != 0:
                f.write("\n")
            for tup in input_data[i]:
                print(tup)
                if (tup[0] == 'START' or tup[0] == 'STOP'):
                    pass
                else:
                    f.write(tup[0] + ' ' + tup[1] + '\n')

    f.close()


output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"
write_dev_out(viterbi(training_file,testing_file),output_path)

# output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\FR\\"
# training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\FR\\train"
# testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\FR\\dev.in"
# write_dev_out(viterbi(training_file,testing_file),output_path)

# output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\CN\\"
# training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\CN\\train"
# testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\CN\\dev.in"
# write_dev_out(viterbi(training_file,testing_file),output_path)

# output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\SG\\"
# training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\SG\\train"
# testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\SG\\dev.in"
# write_dev_out(viterbi(training_file,testing_file),output_path)

('START', None)
('When', 'O')
('I', 'O')
('#UNK#', 'O')
('this', 'O')
('#UNK#', 'O')
(',', 'O')
('I', 'O')
("didn't", 'O')
('think', 'O')
('I', 'O')
('would', 'O')
('be', 'O')
('#UNK#', 'O')
('to', 'O')
('get', 'O')
('in', 'O')
('at', 'O')
('#UNK#', 'O')
(',', 'O')
('but', 'O')
('I', 'O')
('was', 'O')
('#UNK#', 'O')
('to', 'O')
('get', 'O')
('in', 'O')
(',', 'O')
('#UNK#', 'O')
('with', 'O')
('#UNK#', 'O')
('other', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('The', 'O')
('#UNK#', 'O')
('and', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('are', 'O')
('excellent', 'O')
('.', 'O')
('STOP', None)
('START', None)
('My', 'O')
('#UNK#', 'O')
('pizza', 'B-positive')
('#UNK#', 'O')
('in', 'O')
('#UNK#', 'O')
('STOP', None)
('START', None)
('Food', 'B-positive')
('#UNK#', 'O')
(',', 'O')
('its', 'O')
('ok', 'O')
('but', 'O')
('a', 'O')
('bit', 'O')
('#UNK#', 'O')
('for', 'O')
('what', 'O')
('you', 'O')
('get', 'O')
('#UNK#', 'O')
('the', 'O')
('restaurant', 'O')
('#UNK#', 'O')
('a', '

('#UNK#', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('in', 'O')
('Manhattan', 'O')
('STOP', None)
('START', None)
('#UNK#', 'O')
('is', 'O')
('a', 'O')
('little', 'O')
('#UNK#', 'O')
(',', 'O')
('but', 'O')
('to', 'O')
('be', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('However', 'O')
(',', 'O')
('they', 'O')
('do', 'O')
('take', 'O')
('your', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('so', 'O')
('that', 'O')
('you', 'O')
('can', 'O')
('go', 'O')
('#UNK#', 'O')
('out', 'O')
('#UNK#', 'O')
('else', 'O')
('#UNK#', 'O')
('they', 'O')
('#UNK#', 'O')
('you', 'O')
('up', 'O')
('on', 'O')
('your', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('Go', 'O')
('to', 'O')
('#UNK#', 'O')
('sushi', 'B-positive')
('place', 'B-positive')
('or', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('place', 'B-positive')
('-', 'O')
('this', 'O')
('#UNK#', 'O')
('in', 'O')
('#UNK#', 'O')
('is', 'O')
('a', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('Not', 'O')
('one', 'O')
('to', '

('#UNK#', 'O')
('was', 'O')
('#UNK#', 'O')
('of', 'O')
('.', 'O')
('STOP', None)
('START', None)
('#UNK#', 'O')
('a', 'O')
('Japanese', 'O')
('#UNK#', 'O')
(',', 'O')
("I've", 'O')
('#UNK#', 'O')
('in', 'O')
('the', 'O')
('#UNK#', 'O')
('area', 'O')
('for', 'O')
('over', 'O')
('#UNK#', 'O')
('years', 'O')
(',', 'O')
('but', 'O')
('I', 'O')
('was', 'O')
('just', 'O')
('so', 'O')
('#UNK#', 'O')
('at', 'O')
('this', 'O')
('place', 'B-positive')
('.', 'O')
('STOP', None)
('START', None)
('For', 'O')
('the', 'O')
('people', 'O')
('who', 'O')
('want', 'O')
('great', 'O')
('food', 'B-positive')
('#UNK#', 'O')
('great', 'O')
('service', 'B-positive')
(',', 'O')
('#UNK#', 'O')
('is', 'O')
('a', 'O')
('place', 'B-positive')
('to', 'O')
('#UNK#', 'O')
('!', 'O')
('STOP', None)
('START', None)
('#UNK#', 'O')
("don't", 'O')
('#UNK#', 'O')
('us', 'O')
('.', 'O')
('STOP', None)
('START', None)
('#UNK#', 'O')
('down', 'O')
(',', 'O')
('the', 'O')
('best', 'O')
('#UNK#', 'O')
('I', 'O')
('have', 'O')
(